# Técnicas Avançadas de **Captura e Tratamento** de Dados

---
## Prof. Bernardo Alves Furtado
---
### MBA em Big Data, Business Analytics e Gestão de Negócios. @**IDP**
---
3 a 21 agosto  -- 21 horas/aula

## Fonte: Valor 16/8/2021. Manchete:
# Pessimismo da comunidade financeira no Twitter aumenta

![Finwit](https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/Screenshot%20from%202021-08-17%2013-11-05.png?raw=True)
### Ilustração de utilização de prática
### Matéria original: https://valor.globo.com/financas/noticia/2021/08/16/pessimismo-da-comunidade-financeira-no-twitter-aumenta.ghtml


In [ ]:
import requests
url = 'https://queridodiario.ok.org.br/api/'
endpoint = 'gazettes/'

params = {'since': '2020-12-15',
          'until': '2020-12-31',
          'keywords': ['pandemia']}

r = requests.get(f'{url}{endpoint}', params=params).json()
r

In [ ]:
print(r.keys())

In [ ]:
print(type(r['gazettes']))

In [ ]:
print(type(r['gazettes'][0]))

In [ ]:
print(r['gazettes'][0].keys())

In [ ]:
for gazette in r['gazettes']:
    try:
        texto = gazette['file_raw_txt']
    except KeyError:
        pass

texto


In [ ]:
textos = list()
for gazette in r['gazettes']:
    try:
        textos.append(gazette['file_raw_txt'])
    except KeyError:
        pass
textos

---
# Reforçando Captura $+$ Base de Dados
# Exercício II API $+$ SQL



---
1. Seguindo exatamente os passos que utilizamos para ilustrar SQL, vamos simplificar o processo e
criar uma base que atenda os requisitos desta captura de dados de tweets.

2. Conseguem me contar quais passos (em geral) foram esses?

In [ ]:
print('')

1. Vamos começar **CRIANDO** a função de conexão com uma base nova. E o nome da nova base.

In [ ]:
import sqlite3

2. Configuração de dados básicos.

In [ ]:
import os

# Endereço no qual irá guardar a database
tweet_db = 'data/my_first_tweet_database.db'
if not os.path.exists('data'):
    os.mkdir('data')


In [ ]:
# Relembrando dados básicos
from getpass import getpass
bearer_token = getpass()

In [ ]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

twitter_handle = 'folha'
query_params = {'query': f'(from:{twitter_handle})',
                'tweet.fields': 'author_id'}

In [ ]:
def bearer_oauth(bear):
    """
    Method required by bearer token authentication.
    """

    bear.headers["Authorization"] = f"Bearer {bearer_token}"
    bear.headers["User-Agent"] = "v2RecentSearchPython"
    return bear


def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


3. Função genérica de conexão

In [ ]:
def connect_db(database):
    conn = sqlite3.connect(database)
    print(f'Conexão realizada com sucesso.')
    return conn

4. Na sequência, vamos adaptar os comandos **SQL** para a CRIAÇÃO DE TABELAS na base nova.

 * Como exemplo, vamos fazer só uma tabela. Sugiro realizarem de forma mais completa posteriormente.

In [ ]:
sql_create_table = """CREATE TABLE IF NOT EXISTS tweets (
                                id integer PRIMARY KEY,
                                texto text NOT NULL,
                                author_id NOT NULL
                            );"""

5. Com a função de conexão, o nome da base e a instrução para criação da tabela, vamos DE FACTO, criar a tabela.
 * Simplesmente passa o nome da base, executa o comando de criar tabela e *commit* (salva localmente).

In [ ]:
# Rode somente uma vez para criar a table.
with connect_db(tweet_db) as con:
    con.execute(sql_create_table)
    con.commit()

6. Agora a parte distinta do exemplo anterior. Temos que inserir o processo de ATUALIZAÇÃO da base SQL ao processo de consulta da API!
7. Vamos utilizar a mesma função `main` anterior, porém, **alterando**:
 1. No momento de atualização da base.
 2. E na consulta ao valor máximo presente na base.

In [ ]:
def insere_tweet(conn, tweet):
    sql = ''' INSERT INTO tweets(author_id,id,texto)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    # Note que 'tweet' precisa ser recebido como uma lista com sequência de id, author_id e texto
    cur.executemany(sql, tweet)
    conn.commit()
    return cur.lastrowid

In [ ]:
def check_since_id(conn):
    # Note que a condição abaixo é necessária para verificar se há algum item com id na base
    sql = '''SELECT MAX(id) FROM tweets;'''
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchone()[0]

In [ ]:
def main(database):
    # Abre a conexão com SQL
    with connect_db(database) as conn:
        next_token = True
        while next_token:
            # Verificando se o tweet mais recente já está na base
            max_value = check_since_id(con)
            if max_value:
                query_params['since_id'] = max_value
            json_response = connect_to_endpoint(search_url, query_params)
            # Verificando se é necessária mais de uma conexão, ou se já é a última ou única.
            if 'next_token' in json_response['meta']:
                next_token = json_response['meta']['next_token']
                print(next_token)
                query_params['next_token'] = next_token
            else:
                next_token = False
            if 'data' in json_response:
                # Novamente. json_response['data'] é uma lista.
                # Vamos precisar dos dicionários que estão dentro da lista.
                tweets = [list(x.values()) for x in json_response['data']]
                insere_tweet(conn, tweets)

Chama a função com a database

In [ ]:
main(tweet_db)

In [ ]:
# Examinando a base
def collect_basics(conn):
    # Note que a condição abaixo é necessária para verificar se há algum item com id na base
    sql = '''SELECT * FROM tweets LIMIT 5;'''
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
with connect_db(tweet_db) as conn:
    res = collect_basics(conn)

for each in res:
    print(each)

# Aperitivo: análise textual
## Vamos começar com os dados textuais de notícias que obtivemos com a API do Twitter

1. Obtendo a base no pandas
2. Note que usamos `sep=';'` na hora de salvar. Temos que especificá-lo na hora de ler o arquivo.

In [ ]:
import pandas as pd

In [ ]:
address = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/base_tweets_read.csv'
base_texto = pd.read_csv(f'{address}?raw=True', sep=';')

3. Como sempre, vamos investigar os aspectos gerais da base.

In [ ]:
base_texto.info()

In [ ]:
base_texto.describe()

4. Conferindo se não há linhas duplicadas. como `duplicated` retorna False ou True para as linhas.
5. E sabemos que True em python = 1 e False = 0. Se a soma é zero, então...

In [ ]:
sum(base_texto.duplicated())

6. Vamos agora filtrar os tweets para conter apenas as palavras inicialmente.
7. Lembrem.se que o comando `.str` acessa dados das colunas do pandas e disponibiliza as funções de **str**
8. Vamos detalhar mais a questão do <span style="color:red">filtro</span> mais tarde.

In [ ]:
base_texto['words'] = base_texto.text.str.strip().str.split('[\W_]+')

9. Vejam como ficou:

In [ ]:
base_texto.head()

10. Verificando somente uma linha.

In [ ]:
base_texto.loc[0]

11. Dado que transformamos em lista de words, podemos analisar todo o conjunto de palavras de uma vez.
12. Alternativamente, analisaremos cada tweet...

# Tip
### flat_list = [item for sublist in original for item in sublist]

In [ ]:
tweets_words = [item for sublist in base_texto.words for item in sublist]
tweets_words

13. Com a lista completa de palavras, podemos verificar as mais comuns, por exemplo.

In [ ]:
import collections

counter=collections.Counter(tweets_words)
counter

In [ ]:
counter.most_common(10)

In [ ]:
counter.most_common(50)

14. A informação não é assim tão útil, dado que muitas palavras são as chamadas **stop words**
15. Vamos verificar quantas palavras únicas os tweets contêm e também testar se algumas palavras estão presentes na lista.

In [ ]:
palavras_teste = ['medalha', 'CPI', 'frio', 'calor', 'Bolsonaro', 'Lula', 'via', 'hoje', 'futuro']

In [ ]:
for word in palavras_teste:
    try:
        print(f"{word}: {counter[word]}")
    except KeyError:
        print(f'{word} não está na amostra')

16. Bem, conseguimos alguma (vaga) ideia de relevância de tema, por exemplo, uma indicação que talvez tenhamos
 <span style="color:blue">frio </span> mais à frente... ou que talvez hoje seja mais relevante que o futuro,
do ponto de vista jornalístico?


# Noções de Natural Language

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

## Filtering stop words

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
base_texto['tokens'] = base_texto.text.apply(nltk.word_tokenize, language='portuguese')
base_texto.tokens.head()

In [ ]:
base_texto.tokens = base_texto.tokens.apply(lambda w: [x.lower() for x in w if x.lower() not in stopwords])
base_texto.loc[0]

In [ ]:
base_texto.tokens.head()


In [ ]:
fd = nltk.FreqDist(base_texto.tokens.apply(lambda w: w[0]))
fd

In [ ]:
counter=collections.Counter(base_texto.tokens.apply(lambda w: w[0]))
counter.most_common(40)

---
### Noções de
# Regular Expressions -- REGEX
## `import re`

"Regular expressions (called REs, or regexes, or regex patterns) are essentially a tiny, highly specialized programming
language embedded inside Python and made available through the re module. Using this little language,
**you specify the rules for the set of possible strings that you want to match**; this set might contain English
sentences, or e-mail addresses, or TeX commands, or anything you like. You can then ask questions such as
“Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”.
You can also use REs to modify a string or to split it apart in various ways."

source: https://docs.python.org/3/howto/regex.html

## Em partes
1. Linguagem específica, especialista, para <span style="color:red">filtrar</span> texto em grandes porções de texto.
2. Serve para extrair, por exemplo: CPFs, telefones, e-mails, links, etc.
3. Você especifica as REGRAS.
### Não é simples!
4. Depois aplica as regras ao texto e pergunta quais porções satisfazem a regra.
5. Pode **retornar** as porções que satisfazem a regra, ou substituí-las.

* É difícil por que é necessário identificar a função de vários destes "metacharacters" em sequência

# . ^ $ * + ? { } [ ] \ | ( )
---

1. Vamos começar pelo básico. Localizar uma palavra específica.

In [ ]:
import re

pattern = 'gmail'
texto = 'Este é um texto que vamos utilizar para testar se os itens que estamos testando estão aqui dentro deste texto ou não, ' \
        'como disse esse texto pode conter endereços de email do tipo furtadobb@gmail.com, paginas da internet, tais como' \
        'htpps://github.com/bafurtado/PolicySpace2'

re.search(pattern, texto)

2. O resultado é a posição do primeiro item encontrado que corresponde ao padrão, à regra. Vamos conferir

In [ ]:
print(texto[189:194])

3. Pronto, com isso, já conseguimos localizar palavras específicas.
4. Vamos automatizar o processo, usando o objeto python retornado pela `re.search()`

In [ ]:
resultado = re.search(pattern, texto)
resultado.start()

In [ ]:
print(texto[resultado.start():resultado.end()])

---
5. **[ ]** servem para classes de palavras dentro dos colchetes.
6. por exemplo [abc] vai localizar todos **as** os **bs** e os **cs** do texto.

*Na verdade, sem o quantifier que veremos em seguida, o processo identifica apenas o primeiro dos itens.*

In [ ]:
pattern = '[abc]'
re.search(pattern, texto)

8. Conferindo, posição do primeiro a do texto

In [ ]:
print(texto[21])

9. Note que isso é diferente de buscar pela expressão 'abc'.
---

# *
9. Vamos introduzir o asterisco: **\*** primeiro quantifier.
10. O **\*** match <span style="color:red">qualquer número de vezes</span> a expressão anterior.
11. Por exemplo 'ca*t' vai encontrar o que?
12. Pausa.

In [ ]:
print('')

13. Passo-a-passo. o 'c' sempre é necessário. 'a' está antes do **\***, então pode estar incluída 0, 1, 2, 3 ... vezes. O 't' precisa estar no final.

In [ ]:
testes = ['maria', 'nina', 'combat', 'ct', 'cat', 'caaat', 'cast']

In [ ]:
pattern = 'ca*t'
for teste in testes:
    print(f'{teste}: {re.search(pattern, teste)}')


### Exercício

1. E a expressão: 'a[bcd]*b' vai encontrar o que?

In [ ]:
print('')

R. Qualquer palavra que comece com 'a', termine com 'b' e tenha nenhuma letra ou qualquer número de 'b's, 'c's ou 'd's.

In [ ]:
testes = ['adccbb', 'ab', 'axcdb', 'abcxb', 'cat', 'cast']
pattern = 'a[bcd]*b'
for teste in testes:
    print(f'{teste}: {re.search(pattern, teste)}')

---
# $$+$$

14. O $$+$$, por sua vez vai encontrar todas as referências com **pelo menos uma** menção, ou mais
15. 'ca+t', por exemplo vai buscar 'cat', 'caaat', 'caaaaaaat', mas não 'ct'.

---
# ?
16. ? significa uma ou nenhuma vez. Pode ter, ou não ter. Por exemplo sócio-econômico ou socioeconomico, podemos buscar por
*'s?cio?econ?mico'*.

In [ ]:
pattern = 's[oó]?cio-?econ[oô]?mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))

In [ ]:
pattern = 's[oó]+cio-?econ[oô]+mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))

---
# {}
17. Aproveitando o exemplo para introduzir {} que indica um quantifier específico

In [ ]:
pattern = 's[oó]{1}cio-?econ[oô]{1}mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'
texto3 = 'A conjuntura sóocioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))
print(re.search(pattern, texto3))

---
# |
17. De novo, aproveitando o exemplo para introduzir | que indica ou [a|b]
18. Reescrevendo exemplo

In [ ]:
pattern = 's[o|ó]cio-?econ[o|ô]mica'
texto1 = 'A conjuntura sócio-economica.'
texto2 = 'A conjuntura socioeconômica.'
texto3 = 'A conjuntura sóocioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))
print(re.search(pattern, texto3))

---
# {}

19. Para facilitar, os parênteses são utilizados para m (mínimo) e n (máximo) de repetições **da expressão anterior**.
20. 'a{1,3}' significa que a expressão anterior, no caso o 'a', deve ter no mínimo uma vez e no máximo 3.21Portanto, a equivalência abaixo é válida

# *   =   {0, }
# +   =   {1, }
# ?   =   {0, 1}

## exatamente 3 vezes = {3}

---

## Outros detalhes.

21. Para buscar todas as letras minúsculas, pode utilizar o hífen para uma sequência. Por exemplo '[a-z]' serve como regra
para todas as letras minúsculas, como '[0-9]' para todos os dígitos.
22. '[a-zA-Z]', portanto captura tudo que for letra.

# ^

^ é utilizado **NO INÍCIO** de uma sequência como exclusão
23. '[^0-9]' excluirá toda a informação numérica da busca.

---
# Extraindo informação com `re`

## Vimos o processo de 'search'. Mas, também podemos utilizar **re** para extração de informação. Captura.

# ()

24. Se os parênteses fecham uma parte da expressão, apenas aquela expressão será retornada.
25. Ou seja, quando identificamos algum elemento de interesse, podemos capturar apenas uma porção da
informação próxima aquele elemento encontrado. Por exemplo:

In [ ]:
texto3 = 'nome: Bernardo Alves Furtado \n nome: Claudiomar'
pattern = 'nome:{1}(.*)'

re.findall(pattern, texto3)

26. Agora, começamos a complicar. Como previsto.
27. Capturando o e-mail no texto inicial.

In [ ]:
pattern = '([\w]+@[\w]+\.[\w]+)'
re.findall(pattern, texto)

### Passo-a-passo
1. O '(' começa o processo de retornar o que for encontrado dentro, até o próximo ')'.
2. [\w] refere-se a qualquer uma das alternativas entre [a-zA-Z0-9], ou seja a palavra antes do **@**
3. Pelo menos uma letra é necessária. Portanto [\w]+
4. Em seguida, o @, literalmente.
5. Depois outra palavra, após o arroba. Mesma coisa: [\w]+.
6. Depois um ponto. Como o ponto é um caractere especial, precisa ir com o '\\' para ser considerado ponto.
7. Depois, novamente outra palavra [\w]+ e retorna.

### Vamos agora realizar o mesmo exercício, porém com a base de tweets que utilizamos. Vamos extrar todos os links contidos nos tweets.

1. Relendo a base

In [ ]:
import pandas as pd

address = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/base_tweets_read.csv'
base_texto = pd.read_csv(f'{address}?raw=True', sep=';')

base_texto.text.head()

In [ ]:
base_texto.text.loc[0]

2. Nosso padrão, sempre vai conter 'https://'

In [ ]:
pattern = '(https?://[\w./]+)'
re.findall(pattern, base_texto.text.loc[0])

3. Na verdade, no teste, dois links estão disponíveis.
4. O primeiro parece ser o link para a notícia e o segundo é o link do próprio tweet.
5. Vamos investigar sempre os primeiros. Podemos usar, `search()` e capturar o primeiro group, com `.group(0)`

In [ ]:
pattern = '(https://[\w./]+)'
re.search(pattern, base_texto.text.loc[0]).group(0)

6. Para fazer isso para a base toda, vamos utilizar o `re.compile` para ficar mais rápido e utilizar com apply.
7. Confiram que dá no mesmo:

In [ ]:
p = re.compile(pattern)
re.search(p, base_texto.text.loc[0]).group(0)

In [ ]:
# base_texto['links'] = base_texto.text.apply(lambda t: re.search(p, t).group(0))

# O apply simples acima, não funciona quando não há match. Por isso temos que implementar uma condicional.
# Quando não houver, match, add None.

base_texto['links'] = base_texto.text.apply(lambda t: re.search(p, t).group(0) if re.search(p, t) else None)
base_texto.links.head()

In [ ]:
base_texto.links.tail()

In [ ]:
base_texto.info()

# Exercícios

1. Para testes automatizados, com explicação detalhada. Porém, só para o primeiro elemento: *match*, mas não *findall*:

https://regex101.com/r/EwvuHA/47

---
# Outro exemplo de captura por meio de *DataLakes*

## https://basedosdados.org/dataset/mundo-kaggle-olimpiadas
---

1. É necessário instalar basedosdados

## **Caution**
Aqui, e na recomendação da página *basedosdados*, realizamos a instalação via **pip**

Under the hood -- i.e., sem nos avisar, o **pip** desinstalou o `pandas 1.3` corrente e trocou pela versão `1.22`.

Não me lembro, mas alguma função que eu tinha utilizado recentemente só estava disponível na versão `1.3`.

**IRL** -- in real life -- eu só uso `conda envs` no `PyCharm`, com isso consigo controlar quais versões de quais bibliotecas estão instaladas.

*Jupyter* é ótimo para aulas e apresentações, mas não é funcional para *scripts* e programas completos!

2. Base dos Dados possui

# 504

organizações

In [ ]:
# !pip install basedosdados

3. Além de instalar a biblioteca *basedosdados*, para realizar buscas ou ter acesso às bases,
é necessário ter uma conta gratuita (limitada a 10 projetos no gcloud).
4. Você pode se cadastrar de acordo com essas [instruções](https://basedosdados.github.io/mais/access_data_local/).
5. Com isso, e o login na conta do Google (gmail), você terá como preencher corretamente o parâmetro:
`billing_project_id=<nome_do_seu_projeto_cadastrado_no_gcloud>`

In [ ]:
import basedosdados as bd
# Para carregar o dado direto no pandas
df = bd.read_table(dataset_id='mundo_kaggle_olimpiadas',
                   table_id='microdados',
                   billing_project_id='idp-class')

6. Vamos dar uma primeira olhada na base que lemos, direto para uma DataFrame do pandas.

In [ ]:
df.info()

# Tratamento dados

Feita a **Captura**, nos seus vários formatos, é necessário compreender, interpretar, analisar os dados, sejam eles:

1. Numéricos
2. Ou textuais
3. Ou mistos